
---
# **MAGICALLY REVEALING DATA** 
### *Comparing the Number of Character name mentions in the Harry Potter Films*
---

I thought it would be fun if, in some way, the consumption of the data / trends of this visualization had a magical element to it. Remembering the interactive, and specifically layered exposing functionality that altair offered, I set out to make a visualization who's data could disappear and reappear ~magically~

To further get involved with the magical theme, I used a harry potter dataset, which can be found **[here](https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset?select=Harry+Potter+1.csv)**

In [799]:
import pandas as pd
import altair as alt

xls = pd.ExcelFile(
    "./data/Harry Potter 1.xlsx"
)

data = pd.read_excel( xls, "Harry Potter 1" )
chart = alt.Chart(data)


In [800]:
import math

class color:

    hex_values = [ "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F"  ]

    def __init__(self, R, G, B):
        self.R = R
        self.G = G
        self.B = B

    def return_color_between(self, color2, perc):
        r_change = color2.R - self.R
        g_change = color2.G - self.G
        b_change = color2.B - self.B

        r = self.R + (r_change * perc)
        g = self.G + (g_change * perc)
        b = self.B + (b_change * perc)

        return color(r, g, b)

    def return_color_in(self, code):
        if code == "RGB":
            return "{} ({}, {}, {})".format(code, self.R, self.G, self.B)
        if code == "HSB":
            return "{} ({}, {}, {})".format(code, self.R * 360, self.G * 100, self.B * 100)
        if code == "HEX":
            hex1 = self.return_hex(self.R)
            hex2 = self.return_hex(self.G)
            hex3 = self.return_hex(self.B)
            return "#{}{}{}{}{}{}".format( hex1[0], hex1[1], hex2[0], hex2[1], hex3[0], hex3[1] )

    def return_hex(self, component ):
        rounded = math.floor(component / 16)
        remainder = (component / 16) - rounded
        
        hex1 = self.hex_values[rounded]
        hex2 = self.hex_values[math.floor(remainder * 16)]
        return (hex1, hex2)
    
    def return_HSB( self ):
        r = self.R / 255
        g = self.G / 255
        b = self.B / 255

        cmax = max( r, g, b )
        cmin = min( r, g, b )
        delta = cmax - cmin

        h = 0
        s = 0
        v = 0

        # hue:
        if delta == 0:
            h = 0
        elif cmax == r:
            h = 60 * ( (( g - b ) / delta) % 6 )
        elif cmax == g:
            h = 60 * ( (( b - r ) / delta) + 2 )
        elif cmax == b:
            h = 60 * ( (( r - g ) / delta) + 4 )
        
        # saturation
        if cmax == 0:
            s = 0
        else:
            s = delta / cmax
        
        # value
        v = cmax

        return ( h, s * 100, v * 100 )

    def return_color_grad(self, second_color, steps):
        colors = []
        for step in range(0, steps):
            interval = step / (steps - 1)
            color = self.return_color_between(second_color, interval)
            colors.append(color.return_color_in("HEX"))
        return colors

class pallet:
    def __init__(self, grad, primary_color, secondary_color, background, secondary_background, text):
        self.grad = grad
        self.primary_color = primary_color
        self.secondary_color = secondary_color
        self.background = background 
        self.secondary_background = secondary_background
        self.text = text

In [801]:
prim = color( 27, 88, 166)
second = color( 87, 155, 242 )

# second = color( 27, 88, 166)
# prim = color( 87, 155, 242 )

grad = prim.return_color_grad(second, 3)

# back = color(242, 188, 87)
# second_back = color(255, 207, 117)

second_back = color(242, 188, 87)
back = color(255, 207, 117)

text = color( 0, 34, 89 )

prim_pallet = pallet(grad, prim.return_color_in("HEX"), second.return_color_in("HEX"), back.return_color_in("HEX"), second_back.return_color_in("HEX"), text.return_color_in("HEX"))

In [802]:
brush = alt.selection_single(on='mouseover', empty="none", encodings=["x", "y","color"])

In [803]:
def axis_config():
    return alt.Axis(
        domainColor="#FFFFFF00",
        grid=False,
        ticks=False,
        labelColor=prim_pallet.text,
        labelPadding=20,
        labelExpr="datum.value % 1 ? null : datum.label",
        labelFontSize=15,
        titleColor=prim_pallet.text,
        titlePadding=10,
        titleFontSize=20   
    )

In [804]:
scatter_plot = chart.mark_bar(
    cornerRadius=5,
    width=25,
).encode (
    alt.X( 
        "character",
        axis=axis_config(),
    ), alt.Y( 
        "mentions",
        title="# of name mentions in a movie's script",
        axis=axis_config()
    ),
    color=alt.condition(brush, "movie", alt.value("lightgray"), legend=None),
).properties(
    width=100,
)

movies = chart.mark_point( 
    cornerRadius=5,
    size=2500,
    strokeWidth=3,
    stroke=prim_pallet.primary_color,
    color=prim_pallet.primary_color,
    opacity=1,
    fill=prim_pallet.secondary_color,
    filled = True,
    fillOpacity=1
    ).encode (

    alt.Y("movie", 
    scale=alt.Scale(
        domainMax=3,
        domainMin=1
    ), axis=axis_config()
    )).add_selection(
        brush
    )

(scatter_plot | movies).configure(
    font="DINCondensed-Bold",
    background=prim_pallet.background,
    
).configure_view(
    fill=prim_pallet.secondary_background,
    strokeWidth=0
).configure_range(
    ramp=prim_pallet.grad
).configure_legend(
    cornerRadius=10,
    fillColor=prim_pallet.secondary_background,
    labelColor=prim_pallet.text,
    labelFontSize=13,
    padding=10,
    titleFontSize=20
).properties(
    title="Magically revealing the # of Harry Potter character  mentions in a movie",
).configure_title(
    fontSize=30,
    color=prim_pallet.text,
    
)


alt.HConcatChart(...)